<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></div>

In [1]:
import os
import configparser
from tqdm import tqdm
from helpers.helper_functions import *
from helpers.helper_classes import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

c:\Users\caspa\Desktop\data-mining-techniques-vu\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Read config.ini file
config = configparser.ConfigParser()
config.read('src/config.ini')
os.chdir(config['PATH']['ROOT_DIR'])

# # Load data
df = pd.read_parquet(config['PATH']['INT_DIR'] + '/training_set_preprocessed_nodrop.parquet', engine = 'fastparquet')
df_test = pd.read_parquet(config['PATH']['INT_DIR'] + '/test_set_preprocessed_nodrop.parquet', engine = 'fastparquet')
df_mini = df[df['srch_id'] < 10000]


# Bit of feature engineering, TODO: move to preprocessor.py


In [35]:
X_train, X_val, X_test, y_train, y_val, y_test, test_ideal = train_val_test_split(df, 'target', test_size=.2, val_size=.38, random_state=7)

fdsa
X_train_full, X_val_full, y_train_full, y_val_full, _ = train_test_split(df, 'target', test_size=.3)
# y_train_full = df['target']

In [36]:
X_val, desire_df_click = construct_desire(X_val)
X_val, desire_df_book = construct_desire(X_val, target = 'booking_bool')

_, desire_df_click_full = construct_desire(X_val_full)
_, desire_df_book_full = construct_desire(X_val_full, target = 'booking_bool')

X_train = merge_and_drop(X_train, desire_df_click, desire_df_book)
X_test = merge_and_drop(X_test, desire_df_click, desire_df_book)
X_val.drop(['click_bool', 'booking_bool'], axis=1, inplace=True)

X_train_full = merge_and_drop(X_train_full, desire_df_click_full, desire_df_book_full)
df_test = merge_and_drop(df_test, desire_df_click_full, desire_df_book_full, drop=False)
X_val_full.drop(['click_bool', 'booking_bool'], axis=1, inplace=True)

In [24]:
# Testing
# LightGBM ranker
import lightgbm as lgb

# Create dataset
group_train = X_train.groupby('srch_id').size().values
group_val = X_test.groupby('srch_id').size().values
group_test = df_test.groupby('srch_id').size().values

X_train_lgb = X_train.drop(['srch_id'], axis=1)
X_val_lgb = X_test.drop(['srch_id'], axis=1)

params = {
    "objective": "lambdarank",
    "metric":"ndcg",
    'n_estimators': 400,
    'max_depth': 9, 
    'learning_rate': 0.08617902731836785, 
    'subsample': 0.5689847432256938, 
    'colsample_bytree': 0.5808423499586066, 
    'reg_alpha': 0.01702042274150078, 
    'reg_lambda': 0.10839191533155004
}

ranker = lgb.LGBMRanker(**params)

# Training the model
ranker.fit(
      X=X_train_lgb,
      y=y_train,
      group=group_train,
      eval_set=[(X_train_lgb, y_train),(X_val_lgb, y_test)],
      eval_group=[group_train, group_val],
      eval_at=[5])



[1]	training's ndcg@5: 0.38614	valid_1's ndcg@5: 0.378616
[2]	training's ndcg@5: 0.356037	valid_1's ndcg@5: 0.347771
[3]	training's ndcg@5: 0.363681	valid_1's ndcg@5: 0.354947
[4]	training's ndcg@5: 0.369143	valid_1's ndcg@5: 0.358286
[5]	training's ndcg@5: 0.372931	valid_1's ndcg@5: 0.361534
[6]	training's ndcg@5: 0.375631	valid_1's ndcg@5: 0.363537
[7]	training's ndcg@5: 0.378397	valid_1's ndcg@5: 0.366087
[8]	training's ndcg@5: 0.379712	valid_1's ndcg@5: 0.368006
[9]	training's ndcg@5: 0.380399	valid_1's ndcg@5: 0.368415
[10]	training's ndcg@5: 0.381124	valid_1's ndcg@5: 0.368838
[11]	training's ndcg@5: 0.38231	valid_1's ndcg@5: 0.369791
[12]	training's ndcg@5: 0.38299	valid_1's ndcg@5: 0.370697
[13]	training's ndcg@5: 0.384111	valid_1's ndcg@5: 0.370869
[14]	training's ndcg@5: 0.384796	valid_1's ndcg@5: 0.371892
[15]	training's ndcg@5: 0.385424	valid_1's ndcg@5: 0.372263
[16]	training's ndcg@5: 0.386215	valid_1's ndcg@5: 0.373172
[17]	training's ndcg@5: 0.387143	valid_1's ndcg@5: 0

LGBMRanker(colsample_bytree=0.5808423499586066,
           learning_rate=0.08617902731836785, max_depth=9, metric='ndcg',
           n_estimators=350, objective='lambdarank',
           reg_alpha=0.01702042274150078, reg_lambda=0.10839191533155004,
           subsample=0.5689847432256938)

In [32]:
X_train_full

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,month,day,hour,norm_price_usd_srch_id,norm_price_usd_prop_id,norm_price_usd_prop_country_id,norm_price_usd_srch_destination_id,norm_price_usd_srch_length_of_stay,norm_price_usd_srch_booking_window,norm_prop_starrating_srch_id,norm_prop_starrating_prop_id,norm_prop_starrating_prop_country_id,norm_prop_starrating_srch_destination_id,norm_prop_starrating_srch_length_of_stay,norm_prop_starrating_srch_booking_window,norm_prop_review_score_srch_id,norm_prop_review_score_prop_id,norm_prop_review_score_prop_country_id,norm_prop_review_score_srch_destination_id,norm_prop_review_score_srch_length_of_stay,norm_prop_review_score_srch_booking_window,norm_prop_location_score1_srch_id,norm_prop_location_score1_prop_id,norm_prop_location_score1_prop_country_id,norm_prop_location_score1_srch_destination_id,norm_prop_location_score1_srch_length_of_stay,norm_prop_location_score1_srch_booking_window,norm_prop_location_score2_srch_id,norm_prop_location_score2_prop_id,norm_prop_location_score2_prop_country_id,norm_prop_location_score2_srch_destination_id,norm_prop_location_score2_srch_length_of_stay,norm_prop_location_score2_srch_booking_window,rank_price_usd,rank_prop_starrating,rank_prop_review_score,rank_prop_location_score1,rank_prop_location_score2,usd_diff,star_diff,desire_click_bool,desire_booking_bool
0,1,12,187,-1.0,-1.0,219,893,3,3.5,1,2.83,0.0438,4.95,104.77,0,23246,1,0,4,0,1,1,-1.0,-1.00,-1.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,4,4,8,-0.611787,-0.786824,-0.011632,-0.414131,-0.010382,-0.010413,-0.093205,-1.0,0.023546,-0.172030,-0.059211,0.041512,0.016094,-1.0,-0.465867,-0.344933,-0.269834,-0.270610,1.022407,-1.0,0.171391,0.868014,0.166135,0.289655,-0.109967,-0.412219,-0.534749,0.110421,-0.521082,-0.485760,23.5,15.5,18.0,3.5,10.0,-1.0,-1.0,0.037433,0.016043
1,1,12,187,-1.0,-1.0,219,10404,4,4.0,1,2.20,0.0149,5.03,170.74,0,23246,1,0,4,0,1,1,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,4,4,8,0.072867,0.047799,-0.003416,0.528882,-0.004627,-0.007990,1.211671,-1.0,1.045138,1.057051,0.907438,1.039353,0.466718,-1.0,0.142388,0.316967,0.222091,0.248217,-0.192089,-1.0,-0.288586,-0.081617,-0.273203,-0.175353,-0.722035,-0.342981,-0.722099,-0.547866,-0.713093,-0.687435,10.0,5.0,9.5,19.0,19.0,-1.0,-1.0,0.031646,0.025316
2,1,12,187,-1.0,-1.0,219,21315,3,4.5,1,2.20,0.0245,4.92,179.80,0,23246,1,0,4,0,1,1,-1.0,-1.00,-1.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,4,4,8,0.166894,0.032590,-0.002287,0.658391,-0.003837,-0.007657,-0.093205,-1.0,0.023546,-0.172030,-0.059211,0.041512,0.917342,-1.0,0.750644,0.978868,0.714017,0.767044,-0.192089,-1.0,-0.288586,-0.081617,-0.273203,-0.175353,-0.518719,-0.293235,-0.659865,-0.329197,-0.649311,-0.620443,9.0,15.5,3.0,19.0,14.0,-1.0,-1.0,0.006410,0.000000
3,1,12,187,-1.0,-1.0,219,27348,2,4.0,1,2.83,0.0125,4.39,602.77,0,23246,1,0,4,0,1,1,-1.0,-1.00,-1.0,-1.0,-1.0,-1.0,0.0,5.0,-1.0,0.0,5.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,5.0,4,4,8,4.556587,1.701209,0.050392,6.704567,0.033059,0.007881,-1.3

In [33]:
# Testing
# LightGBM ranker
import lightgbm as lgb

# Create dataset
group_train = X_train_full.groupby('srch_id').size().values
# group_val = X_test.groupby('srch_id').size().values
# group_test = df_test.groupby('srch_id').size().values

X_train_lgb = X_train_full.drop(['srch_id'], axis=1)
# X_val_lgb = X_test.drop(['srch_id'], axis=1)

params = {
    "objective": "lambdarank",
    "metric":"ndcg",
    'n_estimators': 400,
    'max_depth': 9, 
    'learning_rate': 0.08617902731836785, 
    'subsample': 0.5689847432256938, 
    'colsample_bytree': 0.5808423499586066, 
    'reg_alpha': 0.01702042274150078, 
    'reg_lambda': 0.10839191533155004
}

ranker = lgb.LGBMRanker(**params)

# Training the model
ranker.fit(
      X=X_train_lgb,
      y=y_train_full,
      group=group_train,
      eval_set=[(X_train_lgb, y_train_full)],
      eval_group=[group_train],
      eval_at=[5])

# Predicting the scores
# test = X_val
test = df_test
test_input = test.drop(['srch_id'], axis=1)
df_res = test


print("Predicting...")
y_pred = ranker.predict(test_input)
df_res['pred_grades'] = y_pred
print("Done predicting")

df_res = df_res.sort_values(by=['srch_id', 'pred_grades'], ascending=[True, False], inplace=False)

df_res
lgbm_submission_desire = df_res[['srch_id', 'prop_id']]
lgbm_submission_desire.to_csv(config['PATH']['SUBMISSION_DIR'] + '/lgbm_submission_desire.csv', index=False)

[1]	training's ndcg@5: 0.300901
[2]	training's ndcg@5: 0.334781
[3]	training's ndcg@5: 0.35103
[4]	training's ndcg@5: 0.360746
[5]	training's ndcg@5: 0.365782
[6]	training's ndcg@5: 0.371033
[7]	training's ndcg@5: 0.374229
[8]	training's ndcg@5: 0.376802
[9]	training's ndcg@5: 0.377582
[10]	training's ndcg@5: 0.378567
[11]	training's ndcg@5: 0.379607
[12]	training's ndcg@5: 0.380438
[13]	training's ndcg@5: 0.38087
[14]	training's ndcg@5: 0.381961
[15]	training's ndcg@5: 0.382955
[16]	training's ndcg@5: 0.383656
[17]	training's ndcg@5: 0.384826
[18]	training's ndcg@5: 0.385484
[19]	training's ndcg@5: 0.385929
[20]	training's ndcg@5: 0.386267
[21]	training's ndcg@5: 0.386751
[22]	training's ndcg@5: 0.38713
[23]	training's ndcg@5: 0.387281
[24]	training's ndcg@5: 0.387827
[25]	training's ndcg@5: 0.387978
[26]	training's ndcg@5: 0.3886
[27]	training's ndcg@5: 0.388887
[28]	training's ndcg@5: 0.389093
[29]	training's ndcg@5: 0.389382
[30]	training's ndcg@5: 0.389967
[31]	training's ndcg@5: 

ValueError: Number of features of the model must match the input. Model n_features_ is 89 and input n_features is 87

In [23]:
import optuna
import lightgbm as lgb

# Create dataset
group_train = X_train.groupby('srch_id').size().values
group_val = X_val.groupby('srch_id').size().values

X_train_lgb = X_train.drop(['srch_id'], axis=1)
X_val_lgb = X_val.drop(['srch_id'], axis=1)

def objective(trial):
    params = {
        "objective": "lambdarank",
        "metric":"ndcg",
        'n_estimators': trial.suggest_int('n_estimators', 100, 500), 
        'max_depth': trial.suggest_int('max_depth', 1, 10), 
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1), 
        'subsample': trial.suggest_float('subsample', 0.4, 0.7), 
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.7), 
        'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 0.2), 
        'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 0.2)
    }

    ranker = lgb.LGBMRanker(**params)

    ranker.fit(
        X=X_train_lgb,
        y=y_train,
        group=group_train
    )

    y_pred = ranker.predict(X_val_lgb)
    df_res = X_val.copy()
    df_res['pred_grades'] = y_pred
    df_res = df_res.sort_values(by=['srch_id', 'pred_grades'], ascending=[True, False], inplace=False)
    df_res = df_res.merge(val_ideal, on=['srch_id', 'prop_id'], how='left')
    ndcg_score = calc_NDCG(val_ideal, df_res)

    return ndcg_score

# Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Extract the best hyperparameters
best_params = study.best_params
print(f'Best hyperparameters: {best_params}')

group_train_full = X_train_full.groupby('srch_id').size().values

# Finally, we can train the model with the best hyperparameters on the whole data
best_ranker = lgb.LGBMRanker(**best_params)
best_ranker.fit(
    X=X_train_full.drop(['srch_id'], axis=1),
    y=y_train_full,
    group=group_train_full,
    eval_set=[(X_train_lgb, y_train)],
    eval_group=[group_train],
    eval_at=[5]
)

# Predict on test set
y_pred = best_ranker.predict(df_test.drop(['srch_id'], axis=1))


[I 2023-05-17 12:33:59,750] A new study created in memory with name: no-name-69bca2b9-4880-471d-bd84-7994d3ab46ca
[W 2023-05-17 12:34:18,446] Trial 0 failed with parameters: {'n_estimators': 373, 'max_depth': 10, 'learning_rate': 0.0327743926070116, 'subsample': 0.6610954599620618, 'colsample_bytree': 0.6065524451996538, 'reg_alpha': 0.19829705311659773, 'reg_lambda': 0.007920390893222856} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\caspa\Desktop\data-mining-techniques-vu\.venv\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\caspa\AppData\Local\Temp\ipykernel_10132\684344144.py", line 26, in objective
    ranker.fit(
  File "c:\Users\caspa\Desktop\data-mining-techniques-vu\.venv\Lib\site-packages\lightgbm\sklearn.py", line 1067, in fit
    super().fit(X, y, sample_weight=sample_weight, init_score=init_score, group=group,

KeyboardInterrupt: 

In [ ]:
# Export best params to txt
with open(config['PATH']['INT_DIR'] + '/best_params.txt', 'w') as f:
    f.write(str(best_params))


## Evaluation


In [ ]:
# Predicting the scores
# test = X_val
test = df_test
test_input = test.drop(['srch_id'], axis=1)
df_res = test


print("Predicting...")
y_pred = best_ranker.predict(test_input)
df_res['pred_grades'] = y_pred
print("Done predicting")

df_res = df_res.sort_values(by=['srch_id', 'pred_grades'], ascending=[True, False], inplace=False)

df_res

Predicting...
Done predicting


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,month,day,hour,norm_price_usd_srch_id,norm_price_usd_prop_id,norm_price_usd_prop_country_id,norm_price_usd_srch_destination_id,norm_price_usd_srch_length_of_stay,norm_price_usd_srch_booking_window,norm_prop_starrating_srch_id,norm_prop_starrating_prop_id,norm_prop_starrating_prop_country_id,norm_prop_starrating_srch_destination_id,norm_prop_starrating_srch_length_of_stay,norm_prop_starrating_srch_booking_window,norm_prop_review_score_srch_id,norm_prop_review_score_prop_id,norm_prop_review_score_prop_country_id,norm_prop_review_score_srch_destination_id,norm_prop_review_score_srch_length_of_stay,norm_prop_review_score_srch_booking_window,norm_prop_location_score1_srch_id,norm_prop_location_score1_prop_id,norm_prop_location_score1_prop_country_id,norm_prop_location_score1_srch_destination_id,norm_prop_location_score1_srch_length_of_stay,norm_prop_location_score1_srch_booking_window,norm_prop_location_score2_srch_id,norm_prop_location_score2_prop_id,norm_prop_location_score2_prop_country_id,norm_prop_location_score2_srch_destination_id,norm_prop_location_score2_srch_length_of_stay,norm_prop_location_score2_srch_booking_window,rank_price_usd,rank_prop_starrating,rank_prop_review_score,rank_prop_location_score1,rank_prop_location_score2,usd_diff,star_diff,pred_grades
23,1,24,216,-1.0,-1.0,219,99484,3,4.0,1,2.40,0.2182,4.54,69.00,1,19222,1,10,2,0,1,0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15,-0.671318,-0.689460,-0.038936,-0.608633,-0.031497,-0.256239,0.435686,-1.0,0.022244,0.423708,-0.06239,-0.152279,-0.018342,-1.0,0.141606,0.056152,0.221727,0.208696,-0.866188,-1.0,-0.146292,-0.225135,-0.134617,-0.266470,1.211751,1.546040,0.596490,1.240322,0.633046,0.570355,20.5,11.5,20.0,22.0,6.0,-1.0,-1.0,1.128720
12,1,24,216,-1.0,-1.0,219,61934,3,4.5,1,2.89,0.2425,4.69,88.88,1,19222,1,10,2,0,1,0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15,-0.133106,-0.446593,-0.031811,-0.023315,-0.025488,-0.194912,0.435686,-1.0,0.022244,0.423708,-0.06239,-0.152279,0.513567,-1.0,0.750018,0.638288,0.713598,0.701668,0.761925,-1.0,0.210782,0.703219,0.207293,0.064784,1.528994,1.688049,0.753974,1.568926,0.793864,0.723864,14.0,11.5,9.5,7.0,1.0,-1.0,-1.0,0.893853
9,1,24,216,-1.0,-1.0,219,54937,3,4.0,1,2.08,0.1649,4.75,83.30,1,19222,1,10,2,0,1,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15,-0.284174,-0.125656,-0.033811,-0.187605,-0.027175,-0.212125,0.435686,-1.0,0.022244,0.423708,-0.06239,-0.152279,-0.018342,-1.0,0.141606,0.056152,0.221727,0.208696,-1.929446,-1.0,-0.379483,-0.831407,-0.357905,-0.482800,0.515907,1.670272,0.251061,0.519556,0.280303,0.233647,16.0,11.5,20.0,28.5,11.0,-1.0,-1.0,0.718350
6,1,24,216,-1.0,-1.0,219,34263,3,4.5,1,3.09,0.1300,4.63,79.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,6.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,6.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15,-0.400588,-0.882482,-0.035352,-0.314208,-0.028475,-0.225390,0.435686,-1.0,0.022244,0.423708,-0.06239,-0.152

In [ ]:

lgbm_submission = df_res[['srch_id', 'prop_id']]
lgbm_submission.to_csv(config['PATH']['SUBMISSION_DIR'] + '/lgbm_submission_optuna.csv', index=False)

In [ ]:
# print(f"RF: {calc_NDCG(test_ideal, pred_ideal_rf)}\n,XGB: {calc_NDCG(test_ideal, pred_xgb_optimized)},\nRandom: {calc_NDCG(test_ideal, pred_random)}")
print(f"XGB: {calc_NDCG(df_ideal, pred_xgb)}, Random: {calc_NDCG(test_ideal, pred_random)}")

XGB: 0.3375755506487008, Random: 0.15050172446700524


## Optuna + XGBRegressor

In [ ]:
# Optimize XGB with optuna
import optuna
from functools import partial

def objective(trial, X_train, y_train, X_test, test_ideal):
    y_train_xgb = y_train.astype(int)
    y_train_xgb[y_train == 5] = 2

    params = {
        "objective": "multi:softprob",
        "random_state": 42,
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 1),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True),
    }

    xgb_model = xgb.XGBClassifier(**params)
    xgb_model.fit(X_train, y_train_xgb)

    pred_xgb = constructs_predictions(xgb_model, X_test, ideal_df=test_ideal)
    ndcg = calc_NDCG(test_ideal, pred_xgb)

    return ndcg

print("Training XGB")
# Assuming you have defined X_train, y_train, X_test, and test_ideal before this point.

# Wrap the objective function with the input data
objective_with_data = partial(objective, X_train=X_train, y_train=y_train, X_test=X_test, test_ideal=test_ideal)

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective_with_data, n_trials=20)




# Train the final model with the best hyperparameters
y_train_xgb = y_train.astype(int)
y_train_xgb[y_train == 5] = 2

best_params = study.best_params
xgb_model_optimized = xgb.XGBClassifier(objective="multi:softprob", random_state=42, **best_params)
xgb_model_optimized.fit(X_train, y_train_xgb)

# Evaluate the optimized model
pred_xgb_optimized = constructs_predictions(xgb_model_optimized, X_test, ideal_df=test_ideal)
pred_xgb_submission = constructs_predictions(xgb_model_optimized, df_test)
print(f"XGB Optimized: {calc_NDCG(test_ideal, pred_xgb_optimized)}")

# pred_submission.to_csv(config['PATH']['DATA_DIR'] + '/submission_RF.csv', index=False)
pred_xgb_submission.to_csv(config['PATH']['DATA_DIR'] + '/submission_XGB.csv', index=False)

[I 2023-05-09 13:56:31,830] A new study created in memory with name: no-name-3a7cbff6-7ed5-4fc1-8c20-9778f5cce14a


Training XGB


[I 2023-05-09 13:58:02,635] Trial 0 finished with value: 0.3422170675116014 and parameters: {'n_estimators': 218, 'max_depth': 10, 'learning_rate': 0.002795642578981349, 'subsample': 0.8932459525721343, 'colsample_bytree': 0.6546014752508442, 'gamma': 0.4545479889258107, 'reg_alpha': 0.0006735472057143736, 'reg_lambda': 0.05659086785788689}. Best is trial 0 with value: 0.3422170675116014.
[I 2023-05-09 13:58:34,110] Trial 1 finished with value: 0.3296238751532763 and parameters: {'n_estimators': 122, 'max_depth': 5, 'learning_rate': 0.00020205115375924383, 'subsample': 0.6995347755906247, 'colsample_bytree': 0.9885228465832642, 'gamma': 0.19381601429279216, 'reg_alpha': 0.03803815623242628, 'reg_lambda': 0.00015357257740569215}. Best is trial 0 with value: 0.3422170675116014.
[I 2023-05-09 13:59:37,760] Trial 2 finished with value: 0.3368665578611753 and parameters: {'n_estimators': 472, 'max_depth': 4, 'learning_rate': 0.004549507912707027, 'subsample': 0.9923851580887791, 'colsample_

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2], got [0 1 5]